# Incremental average

Incremental average

$$s_{n}=\alpha * s_{n-1} + (1-\alpha)*x_{n}$$

for $x_{n}$ numpy array

In [11]:
import numpy as np

class IncrementalAverage():
    def __init__(self, ratio: float=0.1, eps: float=1e-9):
        assert isinstance(ratio, float) and 0.<ratio<=1., 'ratio error, expect float between (0, 1]'
        assert isinstance(eps, float) and 0.<eps<=1., 'eps error, expect float between (0, 1]'
        self._ratio = ratio
        self._eps = eps
        self.reset()
    
    def reset(self):
        self._acc = 0.
        self._divisor = self._eps

    @property
    def value(self):
        return self._acc/(self._divisor + self._eps)
    
    def iterate(self, value: float):
        assert isinstance(value, float), 'value error, expect float, got {}'.format(type(value))
        
        self._acc *= 1-self._ratio
        self._acc += self._ratio*value
        
        self._divisor *= 1-self._ratio
        self._divisor += self._ratio
        
        return self.value

class IncrementalStatistic():
    def __init__(self, ratio: float=0.1, eps: float=1e-9):
        assert isinstance(ratio, float) and 0.<ratio<=1., 'ratio error, expect float between (0, 1]'
        assert isinstance(eps, float) and 0.<eps<=1., 'eps error, expect float between (0, 1]'
        self._avg = IncrementalAverage(ratio, eps)
        self._avg_squ = IncrementalAverage(ratio, eps)
        self._avg_cub = IncrementalAverage(ratio, eps)
        self._eps = eps
    
    def reset(self):
        self._avg.reset()
        self._avg_squ.reset()
        self._avg_cub.reset()
    
    def iterate(self, value: float):
        assert isinstance(value, float), 'value error, expect float, got {}'.format(type(value))
        self._avg.iterate(value)
        self._avg_squ.iterate(value**2)
        self._avg_cub.iterate(value**3)
    
    @property
    def value(self):
        return self._avg.value
    
    @property
    def std(self):
        return np.sqrt(self._avg_squ.value - self._avg.value**2)+self._eps
    
    @property
    def skew(self):
        return (self._avg_cub.value - 3*self._avg_squ.value*self._avg.value + 2*self._avg.value**3)/self.std**3

In [12]:
foo = IncrementalStatistic(ratio=0.001)
for i in range(99):
    if (i % 2) ==0:
        foo.iterate(1.)
    else:
        foo.iterate(2.98)

foo.iterate(3.98)
print(foo._avg.value, foo.std, foo.skew)

2.0009985428688806 1.0056237030764863 0.039035038169475546


In [9]:
foo.skew

0.039035038169475546